In [129]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
# from rf_classification import get_data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from mlxtend.classifier import StackingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import nltk, re, pprint

from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [130]:
data = pd.read_csv('outputt.csv')

In [131]:
data.tail()

,Unnamed: 0,description,remarks,desc_remarks,Transaction_Type,bank_name,label,Sub_Category,Category,mode_of_payment
2402,2813.0,NaN,Via USSD GTBank Transfer 49204402650000005...,sms alert charge for 24july2020-25aug2020,debit,1.0,Bank Transfer Charges,Charges,Commissions,Others
2403,NaN,NaN,NaN,"hhk,bvbvcbnmm,mkjkkjjkhnlkggjk,,nkhhfffjjgvree...",debit,1.0,Invalid Input,Invalid Input,Invalid Input,Invalid Input
2404,NaN,NaN,NaN,"cxxxcnmooofddasujjpopjnk456787hkjdxsxjkl99ynn,,.",credit,1.0,Invalid Input,Invalid Input,Invalid Input,Invalid Input
2405,NaN,NaN,NaN,"zawqsxgbmnklool,mnjjfty99777nbmnmmmmmv90900",credit,1.0,Invalid Input,Invalid Input,Invalid Input,Invalid Input
2406,NaN,NaN,NaN,ccggfgjkjjyrff9864wedtycvnl@@@z44l?8889bvhggfj...,debit,1.0,Invalid Input,Invalid Input,Invalid Input,Invalid Input


In [4]:
data = data.drop(['label', 'Sub_Category', 'mode_of_payment', 'Unnamed: 0', 'description', 'remarks', 'bank_name'], axis = 1)
data

,desc_remarks,Transaction_Type,Category
0,cash wdl from our atm -001570- -079023-gtbank ...,debit,Withdrawal
1,transfer between customers 1100062001091640530...,credit,Salaries & Wages
2,nibss instant payment outward 0000132001091756...,debit,Bill Payments
3,transfer between customers ussd gtbank transfe...,debit,Transfers
4,transfer between customers via gtworld from ik...,credit,Transfer Payments
...,...,...,...
2402,sms alert charge for 24july2020-25aug2020,debit,Commissions
2403,"hhk,bvbvcbnmm,mkjkkjjkhnlkggjk,,nkhhfffjjgvree...",debit,Invalid Input
2404,"cxxxcnmooofddasujjpopjnk456787hkjdxsxjkl99ynn,,.",credit,Invalid Input
2405,"zawqsxgbmnklool,mnjjfty99777nbmnmmmmmv90900",credit,Invalid Input


In [191]:
# transform the data
def get_data(data):
    def clean_data(text):
        new_text = re.sub(r'[^\w\d\s]', '', str(text))
        new_text = re.sub('\s+',' ', str(new_text))
        return new_text.strip().lower()
    
    # process target
    y = data['Category']
    encodeTarget = LabelEncoder()
    Y = encodeTarget.fit_transform(y)

    # process x
    x = data[["Transaction_Type", "desc_remarks"]]
    x["Transaction_Type"] = x["Transaction_Type"].apply(clean_data)
    x["desc_remarks"] = x["desc_remarks"].apply(clean_data)

    # work on desc_remarks
    descRemarks = x['desc_remarks']
    countVecDescRemarks = CountVectorizer(ngram_range=(1,1), # to use bigrams ngram_range=(2,2)
                           stop_words='english')
    descRemarksEncd = countVecDescRemarks.fit_transform(descRemarks)

    # work on Transaction_Type
    transactionType = data['Transaction_Type'].to_numpy().reshape(-1, 1)
    OneHotEnctransType = OneHotEncoder()
    OneHotEnctransType.fit(transactionType)
    transactionTypeEncd = OneHotEnctransType.transform(transactionType)
    return (descRemarksEncd, transactionTypeEncd, Y), (countVecDescRemarks, OneHotEnctransType, encodeTarget)


In [192]:
(descRemarksEncd, transactionTypeEncd, Y), (countVecDescRemarks, OneHotEnctransType, encodeTarget) = get_data(data)

C:\Users\DELL\Anaconda3\new ana\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\DELL\Anaconda3\new ana\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [193]:
transactionTypeEncdArr = transactionTypeEncd.toarray()
transactionTypeEncdArr

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [194]:
descRemarksEncdArr = descRemarksEncd.toarray()
descRemarksEncdArr

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [195]:
X = np.hstack((descRemarksEncdArr, transactionTypeEncdArr))

In [196]:
X


array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [197]:
#X = np.concatenate([descRemarksEncd.toarray(), transactionTypeEncd.toarray()], axis=1)

In [198]:
 # do a quick baseline test
#just checking for the best model
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
baseline = LogisticRegression()
    # # if cv parameter is not specified, default= 3-fold cross validation,
print("CV baseline:", cross_val_score(baseline, X, Y, cv=8).mean())

    # single tree
tree = DecisionTreeClassifier()
print("CV one tree:", cross_val_score(tree, X, Y, cv=8).mean())

model = RandomForestClassifier(n_estimators=200)  # try 10, 20, 50, 100, 200
print("CV forest:", cross_val_score(model, X, Y, cv=8).mean())


svc = LinearSVC(random_state=20)
print("CV svc:", cross_val_score(svc, X, Y, cv=8).mean())


C:\Users\DELL\Anaconda3\new ana\lib\site-packages\sklearn\model_selection\_split.py:668: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=8.
  % (min_groups, self.n_splits)), UserWarning)


CV baseline: 0.9351937984496124


C:\Users\DELL\Anaconda3\new ana\lib\site-packages\sklearn\model_selection\_split.py:668: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=8.
  % (min_groups, self.n_splits)), UserWarning)


CV one tree: 0.9480689368770764


C:\Users\DELL\Anaconda3\new ana\lib\site-packages\sklearn\model_selection\_split.py:668: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=8.
  % (min_groups, self.n_splits)), UserWarning)


CV forest: 0.9497300664451827


C:\Users\DELL\Anaconda3\new ana\lib\site-packages\sklearn\model_selection\_split.py:668: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=8.
  % (min_groups, self.n_splits)), UserWarning)


CV svc: 0.947235603543743


In [199]:
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier

In [200]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [201]:
#using the esemble method to check for the best algorithm
clf = DecisionTreeClassifier()
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
accuracy_score(Y_test, y_pred)

0.9481327800829875

In [202]:
clf = RandomForestClassifier(n_estimators=100, max_features="auto",random_state=20)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
accuracy_score(Y_test, y_pred)

0.950207468879668

In [203]:
clf = ExtraTreesClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=20)
clf.fit(X_train, Y_train)
#scores = cross_val_score(clf, X_train, Y_train, cv=2)
y_pred = clf.predict(X_test)
accuracy_score(Y_test, y_pred)

0.9543568464730291

In [204]:
clf = LinearSVC(random_state=0)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
accuracy_score(Y_test, y_pred)

0.9543568464730291

In [205]:
X_test.shape

(482, 2043)

In [206]:
clf = GradientBoostingClassifier(n_estimators=100)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
accuracy_score(Y_test, y_pred)

0.946058091286307

In [207]:
clf1 = LogisticRegression()
clf2 = RandomForestClassifier(n_estimators=100, max_features="auto",random_state=0)
svc = LinearSVC(random_state=20)
sclf = StackingClassifier(classifiers = [clf1, clf2], use_probas = True, meta_classifier = svc)


In [208]:
for clf, label in zip([clf1, clf2, sclf],
                     ['Linear svc',
                     'Random Forest',
                     'Stacking Classifier']):
    scores = cross_val_score(clf, X_train, Y_train,
                                            cv= 2, scoring = 'f1_macro')
    print(scores.mean(), scores.std(), label)

0.6336434444188547 0.0014052531156336645 Linear svc
0.7489069297021689 0.00035658491569118267 Random Forest
0.7515916681127537 0.001109913400266116 Stacking Classifier


In [209]:
# transform the data
def transform_data(dataFrame=None, countVecDescRemarks=None,OneHotEnctransType=None ):
    def clean_data(text):
        new_text = re.sub(r'[^\w\s]', '', str(text))
        new_text = re.sub('\s+',' ', str(new_text))
        return new_text.strip().lower()

    # process x
    x = dataFrame[["Transaction_Type", "desc_remarks"]]
    x["Transaction_Type"] = x["Transaction_Type"].apply(clean_data)
    x["desc_remarks"] = x["desc_remarks"].apply(clean_data)

    # work on desc_remarks
    descRemarks = x['desc_remarks']
    descRemarksEncd = countVecDescRemarks.transform(descRemarks)

    # work on Transaction_Type
    transactionType = x['Transaction_Type'].to_numpy().reshape(-1, 1)
    transactionTypeEncd = OneHotEnctransType.transform(transactionType)
    return (descRemarksEncd, transactionTypeEncd), (countVecDescRemarks, OneHotEnctransType)


In [112]:
b = ['USSD from 07067821584   TO SKYE(POL)/NURUDEEN RABIU OLAWALE']
b = pd.DataFrame(data = b)
c = ['credit']
c = pd.DataFrame(data = c)
n = np.hstack((b, c))
#n = n.toarray()
n = pd.DataFrame(data = n)
n.columns = ['desc_remarks', 'Transaction_Type']
(descRemarksEncd, transactionTypeEncd), (countVecDescRemarks, OneHotEnctransType) = transform_data(dataFrame=n,countVecDescRemarks=countVecDescRemarks,OneHotEnctransType=OneHotEnctransType)
descRemarksEncdArr = descRemarksEncd.toarray()
transactionTypeEncdArr = transactionTypeEncd.toarray()
xTest = np.hstack((descRemarksEncdArr, transactionTypeEncdArr))

In [210]:
model = DecisionTreeClassifier()
model.fit(X_train, Y_train)

DecisionTreeClassifier()

In [211]:
a = model.predict(xTest)

In [212]:
a
encodeTarget.classes_[a]

array(['Withdrawal'], dtype=object)

In [114]:
b = ['Airtime Purchase USSD-101CT0000000002247823856-234915951678']
b = pd.DataFrame(data = b)
c = ['debit']
c = pd.DataFrame(data = c)
n = np.hstack((b, c))
#n = n.toarray()
n = pd.DataFrame(data = n)
n.columns = ['desc_remarks', 'Transaction_Type']
(descRemarksEncd, transactionTypeEncd), (countVecDescRemarks, OneHotEnctransType) = transform_data(dataFrame=n,countVecDescRemarks=countVecDescRemarks,OneHotEnctransType=OneHotEnctransType)
descRemarksEncdArr = descRemarksEncd.toarray()
transactionTypeEncdArr = transactionTypeEncd.toarray()
xTest = np.hstack((descRemarksEncdArr, transactionTypeEncdArr))

In [116]:
a = model.predict(xTest)
encodeTarget.classes_[a]

array(['Airtime/Data'], dtype=object)

In [119]:
b = ['khdvbjksjkbbkjnk']
b = pd.DataFrame(data = b)
c = ['credit']
c = pd.DataFrame(data = c)
n = np.hstack((b, c))
#n = n.toarray()
n = pd.DataFrame(data = n)
n.columns = ['desc_remarks', 'Transaction_Type']
(descRemarksEncd, transactionTypeEncd), (countVecDescRemarks, OneHotEnctransType) = transform_data(dataFrame=n,countVecDescRemarks=countVecDescRemarks,OneHotEnctransType=OneHotEnctransType)
descRemarksEncdArr = descRemarksEncd.toarray()
transactionTypeEncdArr = transactionTypeEncd.toarray()
xTest = np.hstack((descRemarksEncdArr, transactionTypeEncdArr))

In [120]:
a = model.predict(xTest)
encodeTarget.classes_[a]

array(['Transfer Payments'], dtype=object)

In [121]:
a = model.predict_proba(xTest)

In [122]:
a

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

In [214]:
b = ['Via USSD GTBank Transfer    from AMIDELE OLUWASEFUNMI MARY  BAMIDELE OLUWASEFUNMI MARY to BAMIDELE, I']
b = pd.DataFrame(data = b)
c = ['debit']
c = pd.DataFrame(data = c)
n = np.hstack((b, c))
#n = n.toarray()
n = pd.DataFrame(data = n)
n.columns = ['desc_remarks', 'Transaction_Type']
(descRemarksEncd, transactionTypeEncd), (countVecDescRemarks, OneHotEnctransType) = transform_data(dataFrame=n,countVecDescRemarks=countVecDescRemarks,OneHotEnctransType=OneHotEnctransType)
descRemarksEncdArr = descRemarksEncd.toarray()
transactionTypeEncdArr = transactionTypeEncd.toarray()
xTest = np.hstack((descRemarksEncdArr, transactionTypeEncdArr))

In [215]:
a = model.predict(xTest)
encodeTarget.classes_[a]

array(['Transfers'], dtype=object)

In [216]:
a = model.predict_proba(xTest)
a

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [217]:
b = ['ertyuipkjhgfdsaxcvm']
b = pd.DataFrame(data = b)
c = ['debit']
c = pd.DataFrame(data = c)
n = np.hstack((b, c))
#n = n.toarray()
n = pd.DataFrame(data = n)
n.columns = ['desc_remarks', 'Transaction_Type']
(descRemarksEncd, transactionTypeEncd), (countVecDescRemarks, OneHotEnctransType) = transform_data(dataFrame=n,countVecDescRemarks=countVecDescRemarks,OneHotEnctransType=OneHotEnctransType)
descRemarksEncdArr = descRemarksEncd.toarray()
transactionTypeEncdArr = transactionTypeEncd.toarray()
xTest = np.hstack((descRemarksEncdArr, transactionTypeEncdArr))

In [218]:
a = model.predict(xTest)
encodeTarget.classes_[a]

array(['Withdrawal'], dtype=object)

In [219]:
a = model.predict_proba(xTest)
a

array([[0.        , 0.06716418, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.93283582]])

In [220]:
b = ['NIBSS Instant Payment Outward 000013201226162945000310588550 Swlf TO FBN/SALIU MOSHOOD KOLAWOLE /26.875/REF:00001320122616294500']
b = pd.DataFrame(data = b)
c = ['credit']
c = pd.DataFrame(data = c)
n = np.hstack((b, c))
#n = n.toarray()
n = pd.DataFrame(data = n)
n.columns = ['desc_remarks', 'Transaction_Type']
(descRemarksEncd, transactionTypeEncd), (countVecDescRemarks, OneHotEnctransType) = transform_data(dataFrame=n,countVecDescRemarks=countVecDescRemarks,OneHotEnctransType=OneHotEnctransType)
descRemarksEncdArr = descRemarksEncd.toarray()
transactionTypeEncdArr = transactionTypeEncd.toarray()
xTest = np.hstack((descRemarksEncdArr, transactionTypeEncdArr))

In [221]:
a = model.predict(xTest)
encodeTarget.classes_[a]

array(['Transfer Payments'], dtype=object)

In [224]:
b = ['RVSL*POS/WEB*06-JAN-21*0820993810*00559533']
b = pd.DataFrame(data = b)
c = ['credit']
c = pd.DataFrame(data = c)
n = np.hstack((b, c))
#n = n.toarray()
n = pd.DataFrame(data = n)
n.columns = ['desc_remarks', 'Transaction_Type']
(descRemarksEncd, transactionTypeEncd), (countVecDescRemarks, OneHotEnctransType) = transform_data(dataFrame=n,countVecDescRemarks=countVecDescRemarks,OneHotEnctransType=OneHotEnctransType)
descRemarksEncdArr = descRemarksEncd.toarray()
transactionTypeEncdArr = transactionTypeEncd.toarray()
xTest = np.hstack((descRemarksEncdArr, transactionTypeEncdArr))

In [225]:
a = model.predict(xTest)
encodeTarget.classes_[a]

array(['Transfer Payments'], dtype=object)

In [226]:
b = ['STAMP DUTY CHARGE 11122020 STAMP DUTY CHARGE - 11/12/2020']
b = pd.DataFrame(data = b)
c = ['debit']
c = pd.DataFrame(data = c)
n = np.hstack((b, c))
#n = n.toarray()
n = pd.DataFrame(data = n)
n.columns = ['desc_remarks', 'Transaction_Type']
(descRemarksEncd, transactionTypeEncd), (countVecDescRemarks, OneHotEnctransType) = transform_data(dataFrame=n,countVecDescRemarks=countVecDescRemarks,OneHotEnctransType=OneHotEnctransType)
descRemarksEncdArr = descRemarksEncd.toarray()
transactionTypeEncdArr = transactionTypeEncd.toarray()
xTest = np.hstack((descRemarksEncdArr, transactionTypeEncdArr))

In [227]:
a = model.predict(xTest)
encodeTarget.classes_[a]

array(['Commissions'], dtype=object)

In [228]:
b = ['TRANSFER BETWEEN CUSTOMERS MBANKING - REF:144821621110650780006160736988 ANY Account Transfer from SALIU, MOROHUNMUBO to']
b = pd.DataFrame(data = b)
c = ['debit']
c = pd.DataFrame(data = c)
n = np.hstack((b, c))
#n = n.toarray()
n = pd.DataFrame(data = n)
n.columns = ['desc_remarks', 'Transaction_Type']
(descRemarksEncd, transactionTypeEncd), (countVecDescRemarks, OneHotEnctransType) = transform_data(dataFrame=n,countVecDescRemarks=countVecDescRemarks,OneHotEnctransType=OneHotEnctransType)
descRemarksEncdArr = descRemarksEncd.toarray()
transactionTypeEncdArr = transactionTypeEncd.toarray()
xTest = np.hstack((descRemarksEncdArr, transactionTypeEncdArr))

In [229]:
a = model.predict(xTest)
encodeTarget.classes_[a]

array(['Transfers'], dtype=object)

In [230]:
b = ['TRANSFER BETWEEN CUSTOMERS 090267201114012911626477424013|SENT FROM KUDABANK.COM|090267201114012911626477424013||SALIU, KOLAWOLE']
b = pd.DataFrame(data = b)
c = ['debit']
c = pd.DataFrame(data = c)
n = np.hstack((b, c))
#n = n.toarray()
n = pd.DataFrame(data = n)
n.columns = ['desc_remarks', 'Transaction_Type']
(descRemarksEncd, transactionTypeEncd), (countVecDescRemarks, OneHotEnctransType) = transform_data(dataFrame=n,countVecDescRemarks=countVecDescRemarks,OneHotEnctransType=OneHotEnctransType)
descRemarksEncdArr = descRemarksEncd.toarray()
transactionTypeEncdArr = transactionTypeEncd.toarray()
xTest = np.hstack((descRemarksEncdArr, transactionTypeEncdArr))

In [231]:
a = model.predict(xTest)
encodeTarget.classes_[a]

array(['Transfers'], dtype=object)